<a href="https://colab.research.google.com/github/lucasleonardobs/Inteligent-Systems/blob/main/redes_bayesianas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [101]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/glass/glass.data
!pip -qqq install tqdm
!pip install plotly

--2022-04-19 00:09:29--  https://archive.ics.uci.edu/ml/machine-learning-databases/glass/glass.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11903 (12K) [application/x-httpd-php]
Saving to: ‘glass.data.2’

glass.data.2        100%[===================>]  11.62K  --.-KB/s    in 0s      

2022-04-19 00:09:29 (79.4 MB/s) - ‘glass.data.2’ saved [11903/11903]



# Objetivo do exercício:

* Implementar do zero o pipeline de treinamento em ML.
  * Carregamento dos dados
  * Pré-processamento dos dados
  * Instanciação do modelo  
  * Setup de parâmetros
  * Treinamento 
  * Avaliação do modelo treinado

* Plotar o dataset usando a Análise de componentes Principais (PCA) para redução de dimensionalidade. Responda por escrito em seu notebook:
  * As instâncias de classes diferentes estão visualmente bem separadas?
  * O que essa configuração da distribuição sugere sobre a dificuldade ou facilidade de um classificador qualquer em categorizar corretamente as instâncias?

* O dataset utilizado será o Glass. Seu link está logo abaixo.

* Treinar com duas versões diferentes de Modelo Bayesiano, a seu critério, considerando as implementações do scikit. Justificar o uso dos modelos;

* Utilizar a medida de precisão para avaliar o modelo.

# Lendo a base de dados do glass:

*   Descrição da base: https://archive.ics.uci.edu/ml/datasets/glass+identification

* Colunas:

1. Id number: 1 to 214
2. RI: refractive index
3. Na: Sodium (unit measurement: weight percent in corresponding oxide, as are attributes 4-10)
4. Mg: Magnesium
5. Al: Aluminum
6. Si: Silicon
7. K: Potassium
8. Ca: Calcium
9. Ba: Barium
10. Fe: Iron
11. Type of glass: (class attribute)
-- 1 building_windows_float_processed

-- 2 building_windows_non_float_processed

-- 3 vehicle_windows_float_processed

-- 4 vehicle_windows_non_float_processed (none in this database)

-- 5 containers

-- 6 tableware

-- 7 headlamps

In [102]:
import pandas as pd
from google.colab import data_table

columns = "id,ri,na,mg,ai,si,k,ca,ba,fe,class".split(",")
df = pd.read_csv("glass.data", header=None)
df.columns = columns

data_table.DataTable(df, include_index=False, num_rows_per_page=10)

,id,ri,na,mg,ai,si,k,ca,ba,fe,class
0,1,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.00,0.0,1
1,2,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.00,0.0,1
2,3,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.00,0.0,1
3,4,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.00,0.0,1
4,5,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.00,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...
209,210,1.51623,14.14,0.00,2.88,72.61,0.08,9.18,1.06,0.0,7
210,211,1.51685,14.92,0.00,1.99,73.06,0.00,8.40,1.59,0.0,7
211,212,1.52065,14.36,0.00,2.02,73.42,0.00,8.44,1.64,0.0,7
212,213,1.51651,14.38,0.00,1.94,73.61,0.00,8.48,1.57,0.0,7


# Separando os atributos (features) e a classe

* O campo id (identificador único) deve ser descartado.

In [103]:
X = df.drop(["id", "class"], axis = 1).to_numpy()
y = df["class"].values

# Pré-processando 

## Recodificando os rótulos (classes)

* De maneira geral precisamos mapear as classes de valores textuais para valores numérico-categóricos.
* Para este dataset, entretanto, não é necessário, uma vez que as classes estão apresentadas como números.
* Ou seja, você não precisa fazer nada nesta etapa :)

## Plotando o dataset

* As classes dos elementas deve ser destacadas por cores
* Usamos a técnica de Análise de Componentes Principais para reduzir as dimensões

In [104]:
import pandas as pd
import plotly.express as px

from sklearn.decomposition import PCA


pca = PCA(n_components = 2)
X_plt = pca.fit_transform(X)

df = pd.DataFrame(X_plt)
df["class"] = y
df["class"] = df["class"].astype(str)

px.scatter(df, x=0, y=1, color="class")

## Perguntas:
**As instâncias de classes diferentes estão visualmente bem separadas?**
  
  R:  Não, não existe uma separação clara dos dados do espaço.

**O que essa configuração da distribuição sugere sobre a dificuldade ou facilidade de um classificador qualquer em categorizar corretamente as instâncias?**

  R: Sugere que é difícil mas, não conseguimos inferir isso de imediato visto que os modelos podem capturar padrões que não são perceptíveis quando visualizamos os dados em 2 dimensões.

## Dividindo a base em treino, teste e validação

In [105]:
from sklearn.model_selection import train_test_split


SEED = 0

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state=SEED)
X_test, X_valid, y_test, y_valid = train_test_split(X, y, test_size = 0.50, random_state=SEED)

## Padronizando separadamente as bases

In [106]:
from sklearn.preprocessing import StandardScaler


sc = StandardScaler() # Usado para tentar deixar os dados mais próximos da distribuição gaussiana.

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
X_valid = sc.transform(X_valid)

# Instanciando o modelo



## GaussianNB

In [107]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()

## BernoulliNB


In [108]:
from sklearn.naive_bayes import BernoulliNB

bnb = BernoulliNB()

# Validação do modelo

* Validar testando diferentes valores do *var_smoothing*.
* Considerar o espaço de busca para o *var_smoothing* considerado abaixo. 

## GaussianNB


In [109]:
import warnings
import numpy as np

from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_score


warnings.filterwarnings('ignore')

search_space = np.linspace(1e-12,1, 10)

better_precision = 0
better_var_smoothing = 0

for var_smoothing in search_space:
  gnb = GaussianNB(var_smoothing=var_smoothing)

  gnb.fit(X_train, y_train)

  y_pred = gnb.predict(X_valid)

  precision = precision_score(y_valid, y_pred, average='macro')

  if precision > better_precision:
    better_precision = precision
    better_var_smoothing = var_smoothing

print(f"O parâmetro: {better_var_smoothing} teve a precisão de: {precision}")

O parâmetro: 1e-12 teve a precisão de: 0.4548328816621499


## BernouliNB

In [110]:
import numpy as np

from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import precision_score


bnb = BernoulliNB()

bnb.fit(X_train, y_train)

y_pred = bnb.predict(X_valid)

precision = precision_score(y_valid, y_pred, average='macro')

print(f"A precisão foi de: {precision}")

A precisão foi de: 0.4682395382395382


# Avaliando o modelo final com os parâmetros otimizados

In [111]:
from sklearn.naive_bayes import GaussianNB


gnb = GaussianNB(var_smoothing=better_var_smoothing)

gnb.fit(X_train, y_train)

y_pred = gnb.predict(X_test)

precision = precision_score(y_test, y_pred, average='macro')

print(f"Precisão de: {precision}")

Precisão de: 0.528042328042328


# Quantidade de exemplos por classes

* Buscar na referência das classes scikit como ter acesso às contagens das classes e exibí-las aqui para cada classificador usado


## GaussianNB

In [112]:
label = [1, 2, 3, 5, 6, 7]

class_count = zip(label, gnb.class_count_)

for class_, count in class_count:
  print(f"A classe {class_} apareceu {int(count)} vezes")

A classe 1 apareceu 49 vezes
A classe 2 apareceu 50 vezes
A classe 3 apareceu 10 vezes
A classe 5 apareceu 11 vezes
A classe 6 apareceu 7 vezes
A classe 7 apareceu 22 vezes


## BernoulliNB

In [113]:
label = [1, 2, 3, 5, 6, 7]

class_count = zip(label, bnb.class_count_)

for class_, count in class_count:
  print(f"A classe {class_} apareceu {int(count)} vezes")

A classe 1 apareceu 49 vezes
A classe 2 apareceu 50 vezes
A classe 3 apareceu 10 vezes
A classe 5 apareceu 11 vezes
A classe 6 apareceu 7 vezes
A classe 7 apareceu 22 vezes


# Probabilidade à priori das classes

* A probabilidade à priori equivale à frequência relativa das classes quando não informada através do parâmetro *prios*.

* Buscar na referência das classes scikit como ter acesso às probabilidades à priori calculadas para cada classificar usado.

## GaussianNB

In [114]:
class_prior = zip(label, gnb.class_prior_)

for class_, prior in class_prior:
  print(f"A probabilidade à priori da {class_} é de: {prior}")

A probabilidade à priori da 1 é de: 0.3288590604026846
A probabilidade à priori da 2 é de: 0.33557046979865773
A probabilidade à priori da 3 é de: 0.06711409395973154
A probabilidade à priori da 5 é de: 0.0738255033557047
A probabilidade à priori da 6 é de: 0.04697986577181208
A probabilidade à priori da 7 é de: 0.1476510067114094


## BernoulliNB

In [115]:
class_prior = zip(label, bnb.class_log_prior_)

for class_, prior in class_prior:
  print(f"O log da probabilidade à priori da {class_} é de: {prior}")

O log da probabilidade à priori da 1 é de: -1.1121260078348327
O log da probabilidade à priori da 2 é de: -1.0919233005173132
O log da probabilidade à priori da 3 é de: -2.7013612129514133
O log da probabilidade à priori da 5 é de: -2.6060510331470885
O log da probabilidade à priori da 6 é de: -3.0580361568901457
O log da probabilidade à priori da 7 é de: -1.9129038525871431


# Justificativa do uso dos modelos

O **GaussianNB** foi escolhido inicialmente tendo em vista a natureza dos dados, que se tratam de números reais sem intervalos definidos, o que corresponde perfeitamente com a distribuição normal.

E por mais que o **BernoulliNB** seja voltado para valores binários/categóricos, sendo 0 ou 1 ele foi escolhido por implementar por baixo dos panos uma estratégia que atribue o valor 0 as features que possuem valores negativos e 1 as que possuem positivos.